In [1]:
import sys
sys.path.append("../src")
from imperium.dynamics.auv2d import AUV2D
from imperium.trajectory import Indirect
from imperium.environments.algae import Algae_Farm
import matplotlib.pyplot as plt, numpy as np, pygmo as pg
%matplotlib

Using matplotlib backend: Qt5Agg


In [2]:
# instantiate dynamics
from imperium.dynamics.auv2d import AUV2D
thrust = 1
mass = 10
T = 1000
L = 100
sys = AUV2D(thrust, mass, T, L)

In [3]:
# instantiate trajectory
from imperium.trajectory import Indirect
traj = Indirect(sys)

In [4]:
# create an algae farm
farm = Algae_Farm(5, 5, 2, 6, 10, 40, 40)

# compute waypoint sequence
pts = farm.simple_coverage()

In [6]:
# state bounds
states = [([farm.dsx, farm.dsy, 0, 0], [farm.dsx, farm.dsy, 0, 0])]
for pt in pts:
    states.append(([pt[0], pt[1], -5, -5], [pt[0], pt[1], 5, 5]))
states.append(([farm.dsx, farm.dsy, 0, 0], [farm.dsx, farm.dsy, 0, 0]))
states

[([40, 40, 0, 0], [40, 40, 0, 0]),
 ([11.0, 22.0, -5, -5], [11.0, 22.0, 5, 5]),
 ([5.0, 22.0, -5, -5], [5.0, 22.0, 5, 5]),
 ([5.0, 20.0, -5, -5], [5.0, 20.0, 5, 5]),
 ([11.0, 20.0, -5, -5], [11.0, 20.0, 5, 5]),
 ([11.0, 18.0, -5, -5], [11.0, 18.0, 5, 5]),
 ([5.0, 18.0, -5, -5], [5.0, 18.0, 5, 5]),
 ([5.0, 16.0, -5, -5], [5.0, 16.0, 5, 5]),
 ([11.0, 16.0, -5, -5], [11.0, 16.0, 5, 5]),
 ([11.0, 14.0, -5, -5], [11.0, 14.0, 5, 5]),
 ([5.0, 14.0, -5, -5], [5.0, 14.0, 5, 5]),
 ([5.0, 12.0, -5, -5], [5.0, 12.0, 5, 5]),
 ([11.0, 12.0, -5, -5], [11.0, 12.0, 5, 5]),
 ([11.0, 10.0, -5, -5], [11.0, 10.0, 5, 5]),
 ([5.0, 10.0, -5, -5], [5.0, 10.0, 5, 5]),
 ([5.0, 8.0, -5, -5], [5.0, 8.0, 5, 5]),
 ([11.0, 8.0, -5, -5], [11.0, 8.0, 5, 5]),
 ([11.0, 6.0, -5, -5], [11.0, 6.0, 5, 5]),
 ([5.0, 6.0, -5, -5], [5.0, 6.0, 5, 5]),
 ([40, 40, 0, 0], [40, 40, 0, 0])]

In [ ]:
# set the trajectory
traj.set_state_bounds(states)
traj.set_duration_bounds(1, 500)

In [ ]:
zr = pg.random_decision_vector(*traj.get_bounds())
traj.fitness(zr)

In [ ]:
print(traj.slb, traj.Tlb)
traj.nondimensionalise()
print(traj.slb, traj.Tlb)

In [ ]:
# solve the trajectory
traj.solve()

In [ ]:
# plot trajectory with farm
fig, ax = plt.subplots(1)
farm.plot(ax)
ax.plot(pts.T[0], pts.T[1], "k.")
ax.plot(traj.states[:, 0], traj.states[:, 1], "k.-")

fig.show()

In [ ]:
# show coverage scenario
seg.fitness(pop.champion_x)
fig, ax = pldt.subplots(1)
farm.plot(ax)
pts = farm.simple_coverage()
ax.plot(pts.T[0], pts.T[1], "k.")
ax.plot(seg.states[:, 0], seg.states[:, 1], "k.-")
fig.show()

In [ ]:
zr = pg.random_decision_vector(*traj.get_bounds())
zr

In [ ]:
traj.fitness(zr)

In [ ]:
from multiprocessing import Pool

def f(x):
    return x*x
Pool(5).map(f, [1, 2, 3])

In [ ]:
def mismatch(i):
    return traj.segments[i].mismatch()

In [ ]:
Pool(5).map(mismatch, range(traj.nseg))

In [ ]:
dynamics.__dict__['auv2d']